In [24]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime, Float
import os

In [25]:
load_dotenv()

DB_PASS = os.getenv("DB_PASS")
DB_USER = os.getenv("DB_USER")
DB_HOST = os.getenv("DB_HOST")

engine = create_engine(f"mysql://{DB_USER}:{DB_PASS}@{DB_HOST}/options")
conn = engine.connect()

In [26]:
import pandas as pd

In [27]:
import yahoo_fin.stock_info as si

In [28]:
tickers = pd.read_csv('tickers_csv.csv')['TICKER']

In [29]:
from tqdm import tqdm
data_dict = {}

In [30]:
for ticker in tqdm(tickers):
    pull = si.get_data(ticker=ticker, start_date='8/1/2020', index_as_date=False)
    pull = pull.drop(columns=['open', 'high', 'low', 'close', 'volume'])
    pull['MA'] = pull['adjclose'].rolling(window=30).mean()
    pull['STD'] = pull['adjclose'].rolling(window=30).std()
    pull['Upper'] = pull['MA'] + (pull['STD'] * 2)
    pull['Lower'] = pull['MA'] - (pull['STD'] * 2)
    pull = pull.dropna()
    data_dict[ticker] = pull

100%|██████████| 193/193 [00:25<00:00,  7.47it/s]


In [31]:
data_dict['SHOP'].columns

Index(['date', 'adjclose', 'ticker', 'MA', 'STD', 'Upper', 'Lower'], dtype='object')

In [34]:
data_dict['SHOP']

,date,adjclose,ticker,MA,STD,Upper,Lower
29,2020-09-14,931.830017,SHOP,1019.849335,56.816210,1133.481756,906.216914
30,2020-09-15,929.390015,SHOP,1014.727334,57.839219,1130.405772,899.048895
31,2020-09-16,885.179993,SHOP,1008.233333,61.102716,1130.438765,886.027902
32,2020-09-17,870.760010,SHOP,1000.770333,63.797334,1128.365000,873.175666
33,2020-09-18,901.340027,SHOP,994.544000,64.091677,1122.727355,866.360646
...,...,...,...,...,...,...,...
102,2020-12-28,1147.060059,SHOP,1067.840002,95.943823,1259.727649,875.952356
103,2020-12-29,1171.609985,SHOP,1076.283669,93.443405,1263.170479,889.396859
104,2020-12-30,1163.000000,SHOP,1084.409670,89.844346,1264.098362,904.720978
105,2020-12-31,1131.949951,SHOP,1091.674001,84.227221,1260.128443,923.219559


In [33]:
for ticker in tqdm(tickers):
    data_dict[ticker].to_sql('band_data', conn,
                            if_exists='append', index=False,
                            dtype={
                                'date':DateTime,
                                'adjclose':Float,
                                'ticker':String(10),
                                'MA':Float,
                                'STD':Float,
                                'Upper':Float,
                                'Lower':Float
                            })

100%|██████████| 193/193 [00:18<00:00, 10.60it/s]
